In [ ]:
#Access to my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
import sqlite3

In [ ]:
df = pd.read_csv('drive/MyDrive/SQLpractice/P9-OLTP.csv')
df.head()


,TransactionID,Timestamp,CustomerID,Name,Surname,Shipping_State,Item,Description,Retail_Price,Loyalty_Discount
0,113255,2016-12-13 18:05:37,200000663,Leonard,Jackson,Delaware,8619953091,Pillowcase,18.96,0.02
1,113254,2016-12-13 18:05:22,200000661,Lucas,Abraham,Louisiana,2020397001,Men's Pajama Set,23.10,0.00
2,113253,2016-12-13 18:03:57,200000662,Trevor,Skinner,Louisiana,4681342313,Sheets,29.90,0.10
3,113252,2016-12-13 18:01:45,100000007,Dorothy,Thomson,Alabama,6697166886,Coat,159.80,0.07
4,113251,2016-12-13 18:01:22,400000180,Eric,Bower,New Hampshire,6697166886,Coat,159.80,0.06


In [ ]:
df.shape

(3455, 10)

In [ ]:
unique_values = df.nunique()
unique_values

TransactionID       3455
Timestamp           3434
CustomerID           942
Name                 165
Surname              150
Shipping_State        50
Item                 126
Description           68
Retail_Price         110
Loyalty_Discount      11
dtype: int64

**Note**: Each transaction is unique, it can be concluded that only one item is sold in each transaction

In [ ]:
df.describe()

,TransactionID,CustomerID,Item,Retail_Price,Loyalty_Discount
count,3455.000000,3.455000e+03,3.455000e+03,3455.000000,3455.000000
mean,111528.000000,1.797979e+08,5.276712e+09,58.526237,0.050457
std,997.516917,9.563412e+07,2.600486e+09,34.464217,0.032215
min,109801.000000,1.000000e+08,1.039855e+09,5.600000,0.000000
25%,110664.500000,1.000003e+08,2.963301e+09,31.800000,0.020000
50%,111528.000000,1.000009e+08,5.145202e+09,51.660000,0.050000
75%,112391.500000,2.000009e+08,7.645689e+09,79.800000,0.080000
max,113255.000000,4.000009e+08,9.916068e+09,159.800000,0.100000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3455 entries, 0 to 3454
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TransactionID     3455 non-null   int64  
 1   Timestamp         3455 non-null   object 
 2   CustomerID        3455 non-null   int64  
 3   Name              3455 non-null   object 
 4   Surname           3455 non-null   object 
 5   Shipping_State    3455 non-null   object 
 6   Item              3455 non-null   int64  
 7   Description       3455 non-null   object 
 8   Retail_Price      3455 non-null   float64
 9   Loyalty_Discount  3455 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 270.0+ KB


In [ ]:
#Null values
df.isna().sum()

TransactionID       0
Timestamp           0
CustomerID          0
Name                0
Surname             0
Shipping_State      0
Item                0
Description         0
Retail_Price        0
Loyalty_Discount    0
dtype: int64

**Note** : The dataset contains information on products, clients, and transactions, requiring the creation of three tables to accommodate this information.

## Creating my SQL database

In [ ]:
#Set path to database file in Google Drive
db_path = '/content/drive/MyDrive/SQLpractice/store_sales.db'

In [ ]:
#Create a database for our store sales dataset
conn = sqlite3.connect(db_path)

In [ ]:
#Create table customer
cursor = conn.cursor()
cursor.execute('''CREATE TABLE customer (
  CustomerID INT NOT NULL PRIMARY KEY,
  Name CHAR(30) NOT NULL,
  Surname CHAR(30) NOT NULL,
  Loyalty_Discount FLOAT(1,6) DEFAULT '0.000000'
  ); '''
  )
conn.commit()
cursor.close()

In [ ]:
#Create table item
cursor = conn.cursor()
cursor.execute('''CREATE TABLE item (
  ItemID INT NOT NULL PRIMARY KEY,
  Description VARCHAR(255) NOT NULL,
  Retail_Price FLOAT(9,6) NOT NULL
  ); '''
  )
conn.commit()
cursor.close()

In [ ]:
#Create table transaction
cursor = conn.cursor()
conn.execute('''CREATE TABLE transactions (
  TransactionID INT NOT NULL PRIMARY KEY,
  Timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
  CustomerID INT NOT NULL,
  ItemID INT NOT NULL,
  Shipping_State VARCHAR(255) NOT NULL,
  CONSTRAINT fk_Customer_id FOREIGN KEY (CustomerID) REFERENCES costumer(CostumerID),
  CONSTRAINT fk_Item_id FOREIGN KEY (ItemID) REFERENCES costumer(ItemID)
  ); '''
  )
conn.commit()
cursor.close()

In [ ]:
# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

# Close the cursor and the database connection
cursor.close()

customer
item
transactions


## Formatting data for insertion into the database

In [ ]:
#Changing the name of column Item
df = df.rename(columns={'Item': 'ItemID'})
df.columns

Index(['TransactionID', 'Timestamp', 'CustomerID', 'Name', 'Surname',
       'Shipping_State', 'ItemID', 'Description', 'Retail_Price',
       'Loyalty_Discount'],
      dtype='object')

In [ ]:
#get data to populate table costumer 
customer_data = df[['CustomerID', 'Name', 'Surname', 'Loyalty_Discount']]

In [ ]:
customer_data.shape

In [ ]:
# Making each customer unique
customer_data = customer_data.drop_duplicates(subset=['CustomerID'])

In [ ]:
customer_data.shape

In [ ]:
#get data to populate table item
item_data = df[['ItemID', 'Description', 'Retail_Price']]

In [ ]:
item_data.shape

In [ ]:
#Making each item unique
item_data = item_data.drop_duplicates(subset=['ItemID'])

In [ ]:
item_data.shape

In [ ]:
#get data to populate table transactions
transactions_data = df[['TransactionID', 'Timestamp', 'CustomerID', 'ItemID', 'Shipping_State']]

**Note:** Every transaction is unique

## Populating the database

In [ ]:
#Populate table client
customer_data.to_sql('customer', conn, if_exists='append', index=False)

# Commit changes
conn.commit()

In [ ]:
cursor = conn.cursor()

cursor.execute('SELECT * FROM customer')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

(200000663, 'Leonard', 'Jackson', 0.02)
(200000661, 'Lucas', 'Abraham', 0.0)
(200000662, 'Trevor', 'Skinner', 0.1)
(100000007, 'Dorothy', 'Thomson', 0.07)
(400000180, 'Eric', 'Bower', 0.06)
(200000701, 'Joshua', 'Wilson', 0.1)
(300000467, 'Angela', 'Morrison', 0.08)
(400000706, 'Samantha', 'Mills', 0.1)
(100000061, 'Kylie', 'Howard', 0.1)
(400000513, 'Amy', 'Wilkins', 0.02)
(200000459, 'Charles', 'Brown', 0.04)
(200000482, 'Justin', 'Graham', 0.01)
(100000081, 'Joan', 'Buckland', 0.02)
(200000821, 'Charles', 'Underwood', 0.08)
(300000015, 'Madeleine', 'Marshall', 0.08)
(100000252, 'Nicholas', 'Lyman', 0.09)
(300000822, 'Leonard', 'Fisher', 0.07)
(300000815, 'Pippa', 'Robertson', 0.03)
(100000618, 'Andrew', 'Wright', 0.08)
(100000051, 'Sophie', 'Wilkins', 0.04)
(100000880, 'Rachel', 'Springer', 0.03)
(100000898, 'Samantha', 'Ross', 0.02)
(400000359, 'Gabrielle', 'MacDonald', 0.07)
(100000559, 'Grace', 'Mills', 0.09)
(100000878, 'Felicity', 'Metcalfe', 0.03)
(100000013, 'Ryan', 'MacDonal

In [ ]:
#Populate table item
item_data.to_sql('item', conn, if_exists='append', index=False)

# Commit changes
conn.commit()

In [ ]:
cursor = conn.cursor()

cursor.execute('SELECT * FROM item')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

In [ ]:
#Populate table transactions
transactions_data.to_sql('transactions', conn, if_exists='append', index=False)

# Commit changes
conn.commit()

In [ ]:
cursor = conn.cursor()

cursor.execute('SELECT * FROM transactions')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)